In [3]:
import pickle

with open('data/embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)
    
with open('data/dev.gr.pkl', 'rb') as f:
    dev_gr = pickle.load(f)
    
with open('data/test.gr.pkl', 'rb') as f:
    test_gr = pickle.load(f)

In [9]:
embeddings.shape

(43514, 200)

In [4]:
dev_gr[0].shape

(37480, 11, 50)

In [17]:
test_gr[0].shape, test_gr[1].shape, test_gr[2].shape, test_gr[3].shape, test_gr[4].shape

((38320, 11, 50), (38320, 50), (38320, 50), (38320, 50), (38320,))

In [14]:
!ls -l data

합계 1288100
-rw-rw-r-- 1 kotech kotech  50021962  7월 26 22:05 data.zip
-rw-r--r-- 1 kotech kotech 210188173 10월 20  2021 dev.gr.pkl
-rwxr-xr-x 1 kotech kotech   1353831  2월 19  2019 dev.pkl
-rwxr-xr-x 1 kotech kotech  69622561  2월 14  2019 embeddings.pkl
-rw-rw-r-- 1 kotech kotech        34  7월 26 22:06 readme.md
-rw-rw-r-- 1 kotech kotech      1394  7월 26 22:06 sample_data.txt
-rw-rw-r-- 1 kotech kotech       586  7월 26 22:06 sample_data_english.txt
-rw-rw-r-- 1 kotech kotech       100  7월 26 22:06 sample_vocab.txt
-rw-rw-r-- 1 kotech kotech        90  7월 26 22:06 sample_vocab_english.txt
-rw-r--r-- 1 kotech kotech 214898893 10월 20  2021 test.gr.pkl
-rwxr-xr-x 1 kotech kotech   1384520  2월 19  2019 test.pkl
-rw-r--r-- 1 kotech kotech 765626935 10월 20  2021 train.gr.pkl
-rwxr-xr-x 1 kotech kotech   5871713  2월 15  2019 train.pkl


In [20]:
def sample_fn():
    return 0, 1, 2, 3, 4, 5, 6

(acc, result) = sample_fn()

ValueError: too many values to unpack (expected 2)

In [21]:
import concurrent.futures
import pickle
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence


def multi_sequences_padding(all_sequences, max_sentence_len=50, max_num_utterance=10):
    PAD_SEQUENCE = [0] * max_sentence_len
    padded_sequences = []
    sequences_length = []
    for sequences in all_sequences:
        sequences_len = len(sequences)
        sequences_length.append(get_sequences_length(sequences, maxlen=max_sentence_len))
        if sequences_len < max_num_utterance:
            sequences += [PAD_SEQUENCE] * (max_num_utterance - sequences_len)
            sequences_length[-1] += [0] * (max_num_utterance - sequences_len)
        else:
            sequences = sequences[-max_num_utterance:]
            sequences_length[-1] = sequences_length[-1][-max_num_utterance:]
        sequences = pad_sequences(sequences, padding='post', maxlen=max_sentence_len)
        padded_sequences.append(sequences)
    return padded_sequences, sequences_length


def get_sequences_length(sequences, maxlen):
    sequences_length = [min(len(sequence), maxlen) for sequence in sequences]
    return sequences_length


def generate_data_with_random_samples():
    # generate negative samples randomly
    # In training set, for each sample, we randomly sample a response as a negative candidate
    # In development and test set, for each sample, we randomly sample 9 responses as negative candidates and we add a "EOS" response as a candidate to let model select when to stop
    import random
    import pickle
    vocab = {}
    positive_data = []
    EOS_ID = 7
    with open("./data/sample_vocab.txt", "r", encoding="utf-8") as fr:
        for idx, line in enumerate(fr):
            line = line.strip().split("\t")
            vocab[line[0]] = idx + 1
    with open("./data/sample_data.txt", "r", encoding="utf-8") as fr:
        tmp = []
        for line in fr:
            line = line.strip()
            if len(line) > 0:
                line = line.split("\t")
                if line[0] == "narrative":
                    tmp.append(line[1])
                elif line[0] == "script":
                    tmp.append(line[1])
            else:
                narrative = tmp[0]
                context = tmp[1:]
                narrative_id = [vocab.get(word, 0) for word in narrative.split()]
                context_id = [[vocab.get(word, 0) for word in sent.split()] for sent in context]
                if len(narrative_id) == 0 or len(context_id) == 0:
                    continue
                data = [context_id, narrative_id, 1]
                positive_data.append(data)
                tmp = []
        random.shuffle(positive_data)
        print("all suitable sessions: ", len(positive_data))
        train_num = int(len(positive_data) * 0.9)
        dev_test_num = int(len(positive_data) * 0.05)
        train, dev, test = positive_data[:train_num], positive_data[train_num: train_num + dev_test_num], positive_data[train_num + dev_test_num:]
        train_all, dev_all, test_all = [], [], []
        for context_id, narrative_id, _ in train:
            num_context = len(context_id)
            for i in range(1, num_context):
                context = context_id[:i]
                response = context_id[i]
                train_all.append([context, response, narrative_id, 1])
                flag = True
                while flag:
                    random_idx = random.randint(0, len(positive_data) - 1)
                    random_context = positive_data[random_idx][0]
                    random_idx_2 = random.randint(0, len(random_context) - 1)
                    random_response = random_context[random_idx_2]
                    if len(response) != len(random_response):
                        flag = False
                        train_all.append([context, random_response, narrative_id, 0])
                    else:
                        for idx, wid in enumerate(response):
                            if wid != random_response[idx]:
                                flag = False
                                train_all.append([context, random_response, narrative_id, 0])
                                break
        print(train_all[0], train_all[1])
        for context_id, narrative_id, _ in dev:
            num_context = len(context_id)
            for i in range(1, num_context):
                context = context_id[:i]
                response = context_id[i]
                dev_all.append([context, response, narrative_id, 1])
                count = 0
                negative_samples = []
                while count < 9:
                    random_idx = random.randint(0, len(positive_data) - 1)
                    random_context = positive_data[random_idx][0]
                    random_idx_2 = random.randint(0, len(random_context) - 1)
                    random_response = random_context[random_idx_2]
                    if random_response not in negative_samples and random_response != [EOS_ID]:
                        if len(response) != len(random_response):
                            dev_all.append([context, random_response, narrative_id, 0])
                            count += 1
                            negative_samples.append(random_response)
                        else:
                            for idx, wid in enumerate(response):
                                if wid != random_response[idx]:
                                    dev_all.append([context, random_response, narrative_id, 0])
                                    count += 1
                                    negative_samples.append(random_response)
                                    break
                if response == [EOS_ID]:
                    dev_all.append([context, [EOS_ID], narrative_id, 1])
                else:
                    dev_all.append([context, [EOS_ID], narrative_id, 0])
        print(dev_all[0], dev_all[1], dev_all[2])
        for context_id, narrative_id, _ in test:
            num_context = len(context_id)
            for i in range(1, num_context):
                context = context_id[:i]
                response = context_id[i]
                test_all.append([context, response, narrative_id, 1])
                count = 0
                negative_samples = []
                while count < 9:
                    random_idx = random.randint(0, len(positive_data) - 1)
                    random_context = positive_data[random_idx][0]
                    random_idx_2 = random.randint(0, len(random_context) - 1)
                    random_response = random_context[random_idx_2]
                    if random_response not in negative_samples and random_response != [EOS_ID]:
                        if len(response) != len(random_response):
                            test_all.append([context, random_response, narrative_id, 0])
                            negative_samples.append(random_response)
                            count += 1
                        else:
                            for idx, id in enumerate(response):
                                if id != random_response[idx]:
                                    test_all.append([context, random_response, narrative_id, 0])
                                    negative_samples.append(random_response)
                                    count += 1
                                    break
                if response == [EOS_ID]:
                    test_all.append([context, [EOS_ID], narrative_id, 1])
                else:
                    test_all.append([context, [EOS_ID], narrative_id, 0])
        print(test_all[0], test_all[1], test_all[2])
    context, response, narrative, label = [], [], [], []
    print("train size: ", len(train_all))
    for data in train_all:
        context.append(data[0])
        response.append(data[1])
        narrative.append(data[2])
        label.append(data[3])
    train = [context, response, narrative, label]
    pickle.dump(train, open("./data/train.multi.pkl", "wb"))
    context, response, narrative, label = [], [], [], []
    print("dev size: ", len(dev_all))
    for data in dev_all:
        context.append(data[0])
        response.append(data[1])
        narrative.append(data[2])
        label.append(data[3])
    dev = [context, response, narrative, label]
    pickle.dump(dev, open("./data/dev.multi.pkl", "wb"))
    context, response, narrative, label = [], [], [], []
    print("test size: ", len(test_all))
    for data in test_all:
        context.append(data[0])
        response.append(data[1])
        narrative.append(data[2])
        label.append(data[3])
    test = [context, response, narrative, label]
    pickle.dump(test, open("./data/test.multi.pkl", "wb"))


In [18]:
# generate negative samples randomly
# In training set, for each sample, we randomly sample a response as a negative candidate
# In development and test set, for each sample, we randomly sample 9 responses as negative candidates and we add a "EOS" response as a candidate to let model select when to stop
import random
import pickle
vocab = {}
positive_data = []
EOS_ID = 7
with open("./data/sample_vocab.txt", "r", encoding="utf-8") as fr:
    for idx, line in enumerate(fr):
        line = line.strip().split("\t")
        vocab[line[0]] = idx + 1
with open("./data/sample_data.txt", "r", encoding="utf-8") as fr:
    tmp = []
    for line in fr:
        line = line.strip()
        if len(line) > 0:
            line = line.split("\t")
            if line[0] == "narrative":
                tmp.append(line[1])
            elif line[0] == "script":
                tmp.append(line[1])
        else: # empty line
            narrative = tmp[0]
            context = tmp[1:]
            narrative_id = [vocab.get(word, 0) for word in narrative.split()]
            context_id = [[vocab.get(word, 0) for word in sent.split()] for sent in context]
            if len(narrative_id) == 0 or len(context_id) == 0:
                continue
            data = [context_id, narrative_id, 1]
            positive_data.append(data)
            tmp = []
            print(narrative.split())
            print(data)
    random.shuffle(positive_data)
    print("all suitable sessions: ", len(positive_data))
    #train_num = int(len(positive_data) * 0.9)
    #dev_test_num = int(len(positive_data) * 0.05)
    train_num = 3
    dev_test_num = 1
    train, dev, test = positive_data[:train_num], positive_data[train_num: train_num + dev_test_num], positive_data[train_num + dev_test_num:]
    train_all, dev_all, test_all = [], [], []
    for context_id, narrative_id, _ in train:
        num_context = len(context_id)
        for i in range(1, num_context):
            context = context_id[:i]
            response = context_id[i]
            train_all.append([context, response, narrative_id, 1])
            flag = True
            while flag:
                random_idx = random.randint(0, len(positive_data) - 1)
                random_context = positive_data[random_idx][0]
                random_idx_2 = random.randint(0, len(random_context) - 1)
                random_response = random_context[random_idx_2]
                if len(response) != len(random_response):
                    flag = False
                    train_all.append([context, random_response, narrative_id, 0])
                else:
                    for idx, wid in enumerate(response):
                        if wid != random_response[idx]:
                            flag = False
                            train_all.append([context, random_response, narrative_id, 0])
                            break
    print(train_all[0], train_all[1])
    print(train_all[2], train_all[3])
    print(train_all[4], train_all[5])
    print(train_all[6], train_all[7])
    print(train_all[8], train_all[9])
    print(sdf)
    for context_id, narrative_id, _ in dev:
        num_context = len(context_id)
        for i in range(1, num_context):
            context = context_id[:i]
            response = context_id[i]
            dev_all.append([context, response, narrative_id, 1])
            count = 0
            negative_samples = []
            while count < 9:
                random_idx = random.randint(0, len(positive_data) - 1)
                random_context = positive_data[random_idx][0]
                random_idx_2 = random.randint(0, len(random_context) - 1)
                random_response = random_context[random_idx_2]
                if random_response not in negative_samples and random_response != [EOS_ID]:
                    if len(response) != len(random_response):
                        dev_all.append([context, random_response, narrative_id, 0])
                        count += 1
                        negative_samples.append(random_response)
                    else:
                        for idx, wid in enumerate(response):
                            if wid != random_response[idx]:
                                dev_all.append([context, random_response, narrative_id, 0])
                                count += 1
                                negative_samples.append(random_response)
                                break
            if response == [EOS_ID]:
                dev_all.append([context, [EOS_ID], narrative_id, 1])
            else:
                dev_all.append([context, [EOS_ID], narrative_id, 0])
    print(dev_all[0], dev_all[1], dev_all[2])
    for context_id, narrative_id, _ in test:
        num_context = len(context_id)
        for i in range(1, num_context):
            context = context_id[:i]
            response = context_id[i]
            test_all.append([context, response, narrative_id, 1])
            count = 0
            negative_samples = []
            while count < 9:
                random_idx = random.randint(0, len(positive_data) - 1)
                random_context = positive_data[random_idx][0]
                random_idx_2 = random.randint(0, len(random_context) - 1)
                random_response = random_context[random_idx_2]
                if random_response not in negative_samples and random_response != [EOS_ID]:
                    if len(response) != len(random_response):
                        test_all.append([context, random_response, narrative_id, 0])
                        negative_samples.append(random_response)
                        count += 1
                    else:
                        for idx, id in enumerate(response):
                            if id != random_response[idx]:
                                test_all.append([context, random_response, narrative_id, 0])
                                negative_samples.append(random_response)
                                count += 1
                                break
            if response == [EOS_ID]:
                test_all.append([context, [EOS_ID], narrative_id, 1])
            else:
                test_all.append([context, [EOS_ID], narrative_id, 0])
    print(test_all[0], test_all[1], test_all[2])
context, response, narrative, label = [], [], [], []
print("train size: ", len(train_all))
for data in train_all:
    context.append(data[0])
    response.append(data[1])
    narrative.append(data[2])
    label.append(data[3])
train = [context, response, narrative, label]
pickle.dump(train, open("./data/train.multi.pkl", "wb"))
context, response, narrative, label = [], [], [], []
print("dev size: ", len(dev_all))
for data in dev_all:
    context.append(data[0])
    response.append(data[1])
    narrative.append(data[2])
    label.append(data[3])
dev = [context, response, narrative, label]
pickle.dump(dev, open("./data/dev.multi.pkl", "wb"))
context, response, narrative, label = [], [], [], []
print("test size: ", len(test_all))
for data in test_all:
    context.append(data[0])
    response.append(data[1])
    narrative.append(data[2])
    label.append(data[3])
test = [context, response, narrative, label]
pickle.dump(test, open("./data/test.multi.pkl", "wb"))

['珍妮不喜欢回家。为了陪珍妮，甘决定晚点回家。珍妮是甘最好的朋友']
[[[0], [0], [0], [0], [0], [7]], [0], 1]
['珍妮不喜欢回家。为了陪珍妮，甘决定晚点回家。珍妮是甘最好的朋友']
[[[0], [0], [0], [0], [0], [7]], [0], 1]
['珍妮不喜欢回家。为了陪珍妮，甘决定晚点回家。珍妮是甘最好的朋友']
[[[0], [0], [0], [0], [0], [7]], [0], 1]
['珍妮不喜欢回家。为了陪珍妮，甘决定晚点回家。珍妮是甘最好的朋友']
[[[0], [0], [0], [0], [0], [7]], [0], 1]
['珍妮不喜欢回家。为了陪珍妮，甘决定晚点回家。珍妮是甘最好的朋友']
[[[0], [0], [0], [0], [0], [7]], [0], 1]
all suitable sessions:  5
[[[0]], [0], [0], 1] [[[0]], [7], [0], 0]
[[[0], [0]], [0], [0], 1] [[[0], [0]], [7], [0], 0]
[[[0], [0], [0]], [0], [0], 1] [[[0], [0], [0]], [7], [0], 0]
[[[0], [0], [0], [0]], [0], [0], 1] [[[0], [0], [0], [0]], [7], [0], 0]
[[[0], [0], [0], [0], [0]], [7], [0], 1] [[[0], [0], [0], [0], [0]], [0], [0], 0]


NameError: name 'sdf' is not defined

In [24]:
dev_all

[]

In [25]:
positive_data

[[[[0], [0], [0], [0], [0], [7]], [0], 1],
 [[[0], [0], [0], [0], [0], [7]], [0], 1],
 [[[0], [0], [0], [0], [0], [7]], [0], 1],
 [[[0], [0], [0], [0], [0], [7]], [0], 1]]

In [28]:
train_num = int(len(positive_data) * 0.9)
dev_test_num = int(len(positive_data) * 0.05)

In [27]:
train, dev, test = positive_data[:train_num], positive_data[train_num: train_num + dev_test_num], positive_data[train_num + dev_test_num:]

In [29]:
dev_test_num

0